In [1]:
library(caret)
library(xgboost)
#library(MlBayesOpt)
library(Matrix)
library(rBayesianOptimization)
#https://gitlab.com/avinashbarnwal/elokaggle/blob/master/code/304_hyperparameter_optuna.ipynb
#https://gitlab.com/avinashbarnwal/elokaggle/blob/master/code/302_LGBM_BO_hyperpara.ipynb
#https://cran.r-project.org/web/packages/MlBayesOpt/vignettes/MlBayesOpt.html

Loading required package: lattice

Loading required package: ggplot2



In [2]:
set.seed(1)

In [3]:
data_import =function(dataname){
  filename = paste('https://raw.githubusercontent.com/avinashbarnwal/GSOC-2019/master/AFT/test/data/neuroblastoma-data-master/data/',dataname,'/',sep="")
  inputFileName = paste(filename,'inputs.csv',sep="")
  labelFileName = paste(filename,'outputs.csv',sep="")
  foldsFileName = paste(filename,'cv/equal_labels/folds.csv',sep="")
  inputs        = read.table(inputFileName,sep=",",header=T,stringsAsFactors = F,row.names=1)
  labels        = read.table(labelFileName,sep=",",header=T,stringsAsFactors = F,row.names=1)
  folds         = read.table(foldsFileName,sep=",",header=T,stringsAsFactors = F,row.names=1)
  res           = list()
  res$inputs    = inputs
  res$labels    = labels
  res$folds     = folds
  return(res)
}

In [4]:
data_massage = function(inputs,labels){
    rownamesInput = rownames(inputs)
    inputs        = do.call(data.frame,lapply(inputs, function(x) replace(x, is.infinite(x),NA)))
    naColumns     = colnames(inputs)[colSums(is.na(inputs))>0]
    noVarCol      = getNonVarCols(inputs)
    removeCols    = c(naColumns,noVarCol)
    inputs        = inputs[ , !(colnames(inputs) %in% removeCols)]
    rownames(inputs) = rownamesInput
    labels$min.log.lambda = unlist(lapply(labels$min.log.lambda,exp))
    labels$max.log.lambda = unlist(lapply(labels$max.log.lambda,exp))
    res        = list()
    res$inputs = inputs
    res$labels = labels
    return(res)
}

In [5]:
getXY<-function(foldNo,folds,inputs,labels){
    test.id       = rownames(subset(folds,fold==foldNo))
    train.id      = rownames(subset(folds,fold!=foldNo))
    X             = subset(inputs,rownames(inputs) %in% train.id)
    X             = as.matrix(X)
    X.val         = subset(inputs,rownames(inputs) %in% test.id)
    X.val         = as.matrix(X.val)
    y.label       = subset(labels,rownames(labels) %in% train.id)
    y.label.test  = subset(labels,rownames(labels) %in% test.id)
    y.lower       = as.matrix(y.label$min.log.lambda)
    y.upper       = as.matrix(y.label$max.log.lambda)
    y.lower.val   = as.matrix(y.label.test$min.log.lambda)
    y.upper.val   = as.matrix(y.label.test$max.log.lambda)
    res           = list()
    res$X         = X
    res$X.val     = X.val
    res$y.lower      = y.lower
    res$y.lower.val  = y.lower.val
    res$y.upper      = y.upper
    res$y.upper.val  = y.upper.val
    return(res)
}

In [6]:
getNonVarCols<-function(data){
    var_columns    = apply(inputs,2,var)
    resCol         = names(var_columns[var_columns==0.0])
    return(resCol)
}

In [7]:
# Set Parameters
dataNameRange       = c('ATAC_JV_adipose','CTCF_TDH_ENCODE','H3K27ac-H3K4me3_TDHAM_BP','H3K27ac_TDH_some','H3K36me3_AM_immune')

In [8]:
getaccuracy=function(pred,y_lower,y_higher){
    res = (pred>=y_lower & pred<=y_higher)
    return(res)
}

In [9]:
trainModel = function(X,X_val,y_lower,y_upper,y_lower_val,y_upper_val,param,num_round){
    dtrain = xgb.DMatrix(X)
    setinfo(dtrain,'label_lower_bound', y_lower)
    setinfo(dtrain,'label_upper_bound', y_upper)
    dtest = xgb.DMatrix(X_val)
    setinfo(dtest,'label_lower_bound', y_lower_val)
    setinfo(dtest,'label_upper_bound', y_upper_val)
    watchlist = list(eval = dtest, train = dtrain)
    bst       = xgb.train(param, dtrain, num_round, watchlist,verbose = 0)
    min_val_error = min(bst$evaluation_log$eval)
    return(min_val_error)
    #return(0)
}

In [10]:
xgb_cv_grid = function(max_depth, min_child_weight, subsample, 
                        alpha, lambda, eta, sigma, distribution,X,y.lower,y.upper){

eval_metric  = paste("aft-nloglik@",distribution,",",sigma,sep="")
train.folds  = cut(seq(1,nrow(X)),breaks=5,labels=FALSE)
num_round    = 10000
param = list(eta = eta,
             max_depth = max_depth,
             min_child_weight = min_child_weight,
             subsample = subsample,
             alpha = alpha,
             lambda = lambda,
             aft_sigma = sigma,
             aft_noise_distribution=distribution,
             nthread = 4, 
             verbosity = 0,
             eval_metric = eval_metric,
             objective = "aft:survival")             
res  = list()

return(res)
}

In [1]:
eta              = seq(0.001,1.001,0.1)
max_depth        = seq(2, 10,2)

In [2]:
xgb_grid = expand.grid(eta,max_depth)

In [3]:
xgb_grid

Var1,Var2
<dbl>,<dbl>
0.001,2
0.101,2
0.201,2
0.301,2
0.401,2
0.501,2
0.601,2
0.701,2
0.801,2


In [11]:
nParam = dim(xgb_grid)[1]

ERROR: Error in eval(expr, envir, enclos): object 'xgb_grid' not found


In [13]:
for(i in 1:1){
    res                 = data_import(dataNameRange[1])
    inputs              = res$inputs
    labels              = res$labels
    folds               = res$folds
    resDataMassage      = data_massage(inputs,labels)
    inputs              = resDataMassage$inputs
    labels              = resDataMassage$labels
    fold_iter           = unique(folds$fold)
    accuracy_fold       = numeric(length(fold_iter))
    res                 = getXY(fold_iter[i],folds,inputs,labels)
    X                   = res$X
    X.val               = res$X.val
    y.lower             = res$y.lower
    y.lower.val         = res$y.lower.val
    y.upper             = res$y.upper
    y.upper.val         = res$y.upper.val
    res_loss            = list()
    start_time <- Sys.time()
    for(j in 1:1){
        print(j)
        nrounds   = xgb_grid[j,'nrounds']
        eta       = xgb_grid[j,'eta']
        max_depth = xgb_grid[j,'max_depth']
        min_child_weight =  xgb_grid[j,'min_child_weight']
        subsample        =  xgb_grid[j,'subsample']
        alpha            =  xgb_grid[j,'alpha']
        lambda           =  xgb_grid[j,'lambda']
        sigma            =  xgb_grid[j,'sigma']
        distribution     =  xgb_grid[j,'distribution']
        res_loss[[j]] = xgb_cv_grid(max_depth, min_child_weight, subsample, 
                        alpha, lambda, eta, sigma, distribution,X,y.lower,y.upper)
    }
    end_time <- Sys.time()
    print(end_time - start_time)
}

[1] 1
Time difference of 41.31299 secs


In [ ]:
res

In [ ]:
# cv = xgb.cv(params = list(booster = "gbtree",
#                               max_depth = max_depth,min_child_weight = min_child_weight,
#                               subsample = subsample,alpha = alpha,
#                               lambda = lambda,nrounds = nrounds,
#                               learning_rate = learning_rate,
#                               aft_sigma = sigma,aft_noise_distribution = distribution,
#                               objective = "aft:survival",
#                               nthread = 4,
#                               eval_metric = eval_metric),
#                               data = dtrain,
#                               nfold = 5, prediction = TRUE, showsd = TRUE,
#                               early_stopping_rounds = 10, maximize = TRUE, verbose = 0)
#                               #Score = cv$evaluation_log$test_auc_mean[cv$best_iteration]